In [ ]:
import torch
import os
import cv2
from yolo.utils.utils import *
from predictors.YOLOv3 import YOLOv3Predictor
import glob
from tqdm import tqdm
import sys


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()


#YOLO PARAMS
yolo_df2_params = { "model_def" : "yolo/df2cfg/yolov3-df2.cfg",
"weights_path" : "yolo/weights/yolov3-df2_15000.weights",
"class_path":"yolo/df2cfg/df2.names",
"conf_thres" : 0.5,
"nms_thres" :0.4,
"img_size" : 416,
"device" : device}


#DATASET
dataset = 'deepfashion2' #deepfashion2
yolo_params = yolo_df2_params

#Classes
classes = load_classes(yolo_params["class_path"])

#Colors
cmap = plt.get_cmap("rainbow")
colors = np.array([cmap(i) for i in np.linspace(0, 1, 13)])

model = 'yolo'
detectron = YOLOv3Predictor(params=yolo_params)

while(True):
    path = input('img path: ')
    if not os.path.exists(path):
        print('Img does not exists..')
        continue
    img = cv2.imread(path)
    detections = detectron.get_detections(img)

    
    if len(detections) != 0 :
        detections.sort(reverse=False ,key = lambda x:x[4])
        for x1, y1, x2, y2, cls_conf, cls_pred in detections:

                print("\t+ Label: %s, Conf: %.5f" % (classes[int(cls_pred)], cls_conf))           
                color = colors[int(cls_pred)]
                
                color = tuple(c*255 for c in color)
                color = (.7*color[2],.7*color[1],.7*color[0])       
                    
                font = cv2.FONT_HERSHEY_SIMPLEX   
            
            
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                text =  "%s conf: %.1f" % (classes[int(cls_pred)] ,cls_conf)
                
                cv2.circle(img,(x1,y1),8,(0,0,255),-1)
                cv2.circle(img,(x1,y2),8,(0,0,255),-1)
                cv2.line(img,(x1,y1),(x1,y2),color,5)
                distance = ((((x1 - x1 )**2) + ((y2-y1)**2) )**0.5) 
                print("length:{:}".format(distance))
                
                y1 = 0 if y1<0 else y1
                y1_rect = y1-25
                y1_text = y1-5

                if y1_rect<0:
                    y1_rect = y1+27
                    y1_text = y1+20
                cv2.putText(img,text,(x1,y1_text), font, 0.5,(255,0,0),1,cv2.LINE_AA)
                cv2.putText(img,"length: {:}".format(str(distance)),(x1,y2+20), font, 0.5,(0,0,255),1,cv2.LINE_AA)
                
            
    cv2.imshow('length Detection',cv2.resize(img,(1500,600) ))  
    img_id = path.split('/')[-1].split('.')[0]
    cv2.imwrite('output/ouput-test_{}_{}_{}.jpg'.format(img_id,model,dataset),img)
    
    cv2.waitKey(0)

Model loaded successfully from yolo/weights/yolov3-df2_15000.weights.
img path: tests/11.jpg
	+ Label: short sleeve dress, Conf: 0.54085
length:250.0
	+ Label: short sleeve dress, Conf: 0.55985
length:599.0
	+ Label: shorts, Conf: 0.60323
length:168.0
	+ Label: long sleeve outwear, Conf: 0.72779
length:363.0
	+ Label: long sleeve outwear, Conf: 0.80595
length:355.0
